In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

c:\Users\189\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#загружаем обученную модель и токенизатор
model_path = 'TaskMaster-BERT2'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

In [4]:
#режим проверки (а не тренировки)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [5]:
#функция для обращения к модели
def get_prediction(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    return predicted_class_id

In [15]:
#функция для обозначения класса словом
def id_to_name(prediction):
    if prediction == 0:
        predicted_name = 'home'
    elif prediction == 1:
        predicted_name = 'work-study'
    elif prediction == 2:
        predicted_name = 'shopping'
    return predicted_name

In [24]:
text = "организовать корпоратив"
prediction = get_prediction(text)
tag = id_to_name(prediction)
print("Predicted class:", prediction, "Predicted name:", tag)

Predicted class: 0 Predicted name: home
